In [ ]:
!pip install Faker

In [46]:
import random
import time
import requests
from bs4 import BeautifulSoup
from faker import Faker
import csv

base_url = "https://www.amazon.com/dp/"


# headers = {
#           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
#           'Accept-Language': 'da, en-gb, en',
#           'Accept-Encoding': 'gzip, deflate, br',
#           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
#           'Referer': 'https://www.google.com/'
#         }


def get_soup_retry(url):
    fake = Faker()
    uag_random = fake.user_agent()

    header = {
        'User-Agent': uag_random,
        'Accept-Language': 'en-US,en;q=0.9'
    }
    isCaptcha = True
    while isCaptcha:
        time.sleep(random.uniform(0.5, 1))
        page = requests.get(url, headers=header)
        if not page.status_code == 200:
            print(f"page status code error: {page.status_code}")
        soup = BeautifulSoup(page.content, 'html.parser')
        if 'captcha' in str(soup):
            uag_random = fake.user_agent()
            print(f'\rBot has been detected... retrying ... use new identity: {uag_random} ', end='', flush=True)
            continue
        else:
            print('Bot bypassed')
            return soup, page.status_code


def get_description(soup):
    div_class_description = "a-expander-content a-expander-partial-collapse-content"
    outer_div = soup.find('div', {'data-a-expander-name': 'book_description_expander'})
    
    if outer_div:
        inner_div = outer_div.find('div', class_=div_class_description)
        if inner_div:
            inner_text = inner_div.get_text(strip=True, separator=' ')
            return inner_text
        else:
            return None
    else:
        return None
    
def get_ages(soup):
    #TODO
    return None

def get_new_values(url):
    soup, status_code = get_soup_retry(url)
    
    description = get_description(soup)
    if description is None:
        print(f"Description not found for url: {url}")
        description = ""
    ages = get_ages(soup)
    if ages is None:
        print(f"Ages not found for url: {url}")
        ages = "None"
    return {"description" : description, "ages" : ages, "code" : status_code}
    

def process_csv_no_header(input_csv, output_csv, start_row, end_row):
    with open(input_csv, mode='r', encoding='utf-8', errors='ignore') as infile, open(output_csv, mode='w', encoding='utf-8', newline='') as outfile:
        # Read input as plain rows
        reader = csv.reader(infile, quotechar='"')
        writer = csv.writer(outfile, quotechar='"', quoting=csv.QUOTE_MINIMAL) # make sure to correctly escape characters

        for row_index, row in enumerate(reader):
            # Process rows within the range
            if start_row <= row_index < end_row:
                if len(row) >= 7:
                    # Check if there is an 8th, 9th and 10th field for description, ages and status_code
                    if len(row) >= 9:
                        description = row[7].strip()
                        ages = row[8].strip()
                        # Replace description and ages only if one of them is empty
                        if not description or not ages:
                            asin = row[0]  # Assuming ASIN is in the first column
                            new_values = get_new_values(base_url + asin)
                            row[7] = new_values["description"]
                            row[8] = new_values["ages"]
                            row[9] = new_values["code"]
                    else:
                        # Add a new 8th and 9th field if not present
                        asin = row[0]  # Assuming ASIN is in the first column
                        new_values = get_new_values(base_url + asin)
                        row.append(new_values["description"])
                        row.append(new_values["ages"])
                        row.append(new_values["code"])
            # Write the row to the output CSV
            writer.writerow(row)


In [47]:
input_csv_path = "book32-listing.csv"
output_csv_path = "output_with_new_values.csv"

# Specify range of rows to process (e.g., rows 0–10)
process_csv_no_header(input_csv_path, output_csv_path, start_row=0, end_row=10)

Bot has been detected... retrying ... use new identity: Mozilla/5.0 (iPod; U; CPU iPhone OS 3_3 like Mac OS X; kw-GB) AppleWebKit/531.25.4 (KHTML, like Gecko) Version/3.0.5 Mobile/8B118 Safari/6531.25.4  page status code error: 404
Bot bypassed
Description not found for url: https://www.amazon.com/dp/761183272
Ages not found for url: https://www.amazon.com/dp/761183272
Bot has been detected... retrying ... use new identity: Mozilla/5.0 (iPad; CPU iPad OS 1_1_5 like Mac OS X) AppleWebKit/533.1 (KHTML, like Gecko) CriOS/22.0.839.0 Mobile/72T285 Safari/533.1 Bot bypassed
Ages not found for url: https://www.amazon.com/dp/1623439671
Bot has been detected... retrying ... use new identity: Mozilla/5.0 (Windows; U; Windows NT 5.0) AppleWebKit/535.49.4 (KHTML, like Gecko) Version/5.0 Safari/535.49.4 ile/95R543 Safari/536.1 ri/6531.50.5 Bot bypassed
Description not found for url: https://www.amazon.com/dp/B00O80WC6I
Ages not found for url: https://www.amazon.com/dp/B00O80WC6I
Bot has been detect